In [4]:
import json
import pandas as pd
import numpy as np

In [5]:
business = []
for line in open('./yelp_dataset/business.json', 'r'):
    business.append(json.loads(line))

In [3]:
# len(business)

192609

In [6]:
bus_json = {"business":business}
df = pd.DataFrame(bus_json['business'])

In [7]:
test = df['categories'].to_numpy()
number = set()
idxs = []
for i in range(len(df)):
    sen = df.iloc[i]
    if sen['categories'] is None:
        continue
    elif 'Restaurants' in sen['categories'] and 'Food' in sen['categories']:
        number.update(sen['categories'].split(", "))
        idxs.append(sen.name)
    
rest_data = df.iloc[idxs]

In [14]:
cat_list = list(number)
cat_list.remove('Restaurants')
cat_list.remove('Food')
cat_one_hot = np.zeros((len(rest_data),len(cat_list)))
for i in range(len(rest_data)):
    l = rest_data.iloc[i]
    c_col = l['categories']
    if c_col != None:
        cats = [c for c in c_col.split(', ')]
        for c in cats:
            if c != 'Restaurants' and c != 'Food':
                cat_one_hot[i][cat_list.index(c)] = 1

In [48]:
# cat_rest = dict()
# for l in df['categories']:
#     if l != None:
#         cats = [c for c in l.split(', ')]
#         if 'Restaurants' in cats and 'Food' in cats:
#             for c in cats:
#                 if c in cat_rest:
#                     cat_rest[c]+= 1
#                 else:
#                     cat_rest[c] = 1

In [60]:
# cat_rest_dedup = dict()
# for i in range(len(df)):
#     l = df.iloc[i]
#     c_col = l['categories']
#     if c_col != None:
#         cats = [c for c in c_col.split(', ')]
#         if 'Restaurants' in cats and 'Food' in cats:
#             for c in cats:
#                 if c in cat_rest_dedup:
#                     cat_rest_dedup[c].add(i)
#                 else:
#                     cat_rest_dedup[c] = set()
#                     cat_rest_dedup[c].add(i)

In [42]:
class YelpData:
    def __init__(self):
        self.rest_data = None
        self.categories = None
        self.ratings = None
        self.cat_one_hot = None
        

    def process(self):
        # load json file to dataframe
        business = []
        for line in open('./yelp_dataset/business.json', 'r'):
            business.append(json.loads(line))
        bus_json = {"business":business}
        df = pd.DataFrame(bus_json['business'])

        # extract restaurants
        number = set()
        idxs = []
        for i in range(len(df)):
            sen = df.iloc[i]
            if sen['categories'] is None:
                continue
            elif 'Restaurants' in sen['categories'] and 'Food' in sen['categories']:
                number.update(sen['categories'].split(", "))
                idxs.append(sen.name)
        rest_data = df.iloc[idxs]
        cat_list = list(number)

        # one hot encoder
        cat_one_hot = np.zeros((len(rest_data),len(cat_list)))
        for i in range(len(rest_data)):
            l = rest_data.iloc[i]
            c_col = l['categories']
            if c_col != None:
                cats = [c for c in c_col.split(', ')]
                for c in cats:
                    cat_one_hot[i][cat_list.index(c)] = 1

        self.rest_data = rest_data
        self.categories = cat_list
        self.ratings = rest_data['stars']
        self.cat_one_hot = cat_one_hot
        
    def add_bias(self):
        cat_one_hot_bias = np.ones((self.cat_one_hot.shape[0],self.cat_one_hot.shape[1]+1))
        for i in range(len(cat_one_hot_bias)):
            encoded_cats = self.cat_one_hot[i]
            cat_one_hot_bias[i][:len(encoded_cats)] = encoded_cats
        return cat_one_hot_bias

In [25]:
# adding feature test
import json
import pandas as pd
import numpy as np
import pickle

class YelpData:
    def __init__(self):
        self.rest_data = None
        self.categories = None
        self.ratings = None
        self.cat_one_hot = None
        self.index_length = None
        self.cities_to_include = ['Toronto', 'Las Vegas', ' Phoenix', 'Charlotte' ,'Calgary', 'Montréal','Pittsburgh', 'Scottsdale','Cleveland','Mesa']

    def process(self):
        # load json file to dataframe
        business = []
        for line in open('./yelp_dataset/business.json', 'r'):
            business.append(json.loads(line))
        bus_json = {"business":business}
        df = pd.DataFrame(bus_json['business'])

        # extract restaurants
        cities_num = {}
        cities = set()
#         self.codes = set()
        number = set()
        idxs = []
        for i in range(len(df)):
            sen = df.iloc[i]
            if sen['categories'] is None or sen['city'] not in self.cities_to_include:
                continue
            elif 'Restaurants' in sen['categories'] and 'Food' in sen['categories']:
                number.update(sen['categories'].split(", "))
                cities.add(sen['city'])
#                 if sen['city'] in cities_num:
#                     cities_num[sen['city']] += 1
#                 else:
#                     cities_num[sen['city']] = 1
#                 codes.add(sen['postal_code'])
                idxs.append(sen.name)
        rest_data = df.iloc[idxs]
        cat_list = list(cities) + list(number)
        cat_list.remove('Restaurants')
        cat_list.remove('Food')
        self.index_length = len(cat_list)
        
        word_to_index = {w:i for i,w in enumerate(cat_list)}
        # one hot encoder
        cat_one_hot = np.zeros((len(rest_data), self.index_length))
        for i in range(len(rest_data)):
            put_city = False
            l = rest_data.iloc[i]
            c_col = l['categories']
            if c_col != None:
                cats = c_col.split(', ')
#                 cats = [c for c in c_col.split(', ')]
                for c in cats:
                    if c != 'Restaurants' and c != 'Food':
                        cat_one_hot[i][word_to_index[c]] = 1
                        put_city =True
                if put_city:
                    cat_one_hot[i][word_to_index[l['city']]]
                
                
        self.cities_num = cities_num
        self.rest_data = rest_data
        self.categories = cat_list
#         self.cat_index = {j:i for i,j in enumerate(cat_list)}
        self.ratings = rest_data['stars']
        self.cat_one_hot = cat_one_hot
        
    def add_bias(self):
        bias = np.ones([self.cat_one_hot.shape[0],1])
        cat_one_hot_bias = np.concatenate([self.cat_one_hot, bias], axis = 1)
#         cat_one_hot_bias = np.ones((self.cat_one_hot.shape[0],self.cat_one_hot.shape[1]+1))
#         for i in range(len(cat_one_hot_bias)):
#             encoded_cats = self.cat_one_hot[i]
#             cat_one_hot_bias[i][:len(encoded_cats)] = encoded_cats
        return cat_one_hot_bias
    
    def to_pickle(self, data, name):
        with open(name, "wb") as f:
            pickle.dump(data, f)
    
    def load(self, name):
        with open(name, "rb") as f:
            data = pickle.load(f)
        return data
    
    def get_vector(self, cats, index_list = None):
        if index_list:
            res = np.zeros([1, len(index_list) + 1])
            for w in cats:
                res[0][index_list[w]] = 1

        else:
            res = np.zeros([1,self.index_length + 1])
            for w in cats:
                res[0][self.cat_index[w]] = 1
        #adding bias
        res[0][-1] = 1
        return res

In [26]:
df = YelpData()
df.process()
# cat_one_hot_bias = dt.add_bias()

In [29]:
res.shape

(10782, 470)

In [18]:
nums = [[v,k] for k, v in df.cities_num.items()]

In [19]:
nums.sort()

In [20]:
nums.reverse()

In [16]:
cities_to_include = ['Toronto', 'Las Vegas', ' Phoenix', 'Charlotte' ,'Calgary', 'Montréal','Pittsburgh', 'Scottsdale','Cleveland','Mesa']

12446

In [21]:
pyro.clear_store_param()

[[2656, 'Toronto'],
 [2615, 'Las Vegas'],
 [1664, 'Phoenix'],
 [1094, 'Charlotte'],
 [996, 'Calgary'],
 [992, 'Montréal'],
 [842, 'Pittsburgh'],
 [577, 'Scottsdale'],
 [535, 'Cleveland'],
 [475, 'Mesa'],
 [467, 'Mississauga'],
 [455, 'Madison'],
 [416, 'Tempe'],
 [375, 'Henderson'],
 [324, 'Chandler'],
 [306, 'Glendale'],
 [294, 'Markham'],
 [271, 'Gilbert'],
 [187, 'Brampton'],
 [171, 'Scarborough'],
 [165, 'Champaign'],
 [163, 'Vaughan'],
 [154, 'North Las Vegas'],
 [153, 'Richmond Hill'],
 [148, 'Peoria'],
 [139, 'North York'],
 [127, 'Concord'],
 [106, 'Etobicoke'],
 [93, 'Surprise'],
 [83, 'Avondale'],
 [76, 'Mentor'],
 [76, 'Goodyear'],
 [74, 'Gastonia'],
 [72, 'Matthews'],
 [70, 'Laval'],
 [70, 'Lakewood'],
 [67, 'Fort Mill'],
 [63, 'Newmarket'],
 [62, 'Akron'],
 [62, 'Ajax'],
 [58, 'Oakville'],
 [55, 'Huntersville'],
 [54, 'Urbana'],
 [53, 'Thornhill'],
 [49, 'Strongsville'],
 [49, 'Pickering'],
 [49, 'Aurora'],
 [48, 'Cuyahoga Falls'],
 [44, 'Stow'],
 [44, 'Medina'],
 [43, 'Pa